In [1]:
!pip install spacy
!pip install sklearn
! python -m spacy download en_core_web_sm


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 89.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
 #First, install the necessary libraries
!pip install fuzzywuzzy
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.4 MB/s eta 0:00:00


In [3]:
pip install pandas langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=6435340b0aebe63ffa91fd3a0bdea62fdba6bd7d9610051b29b2f0a52592b303
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


**Step 1: Data Cleaning**

#First, import the necessary libraries

In [4]:
import pandas as pd
import requests
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
from textblob import TextBlob
from langdetect import detect, LangDetectException
import re

#load data

First, we need to load the CSV data, inspect it, and clean it for further analysis

In [6]:
# Load the dataset
data = pd.read_csv('/content/sample_data/disaster tweets.csv')

# Display the first few rows of the dataset
print(data.head())

                                            Name            Handle  \
0                                 Eco Discussion    @EcoDiscussion   
1  Daily News - Trending | Breaking | Technology      @mirror7news   
2                                         Shohag  @MdShohi70476789   
3                                 City Headlines   @CityHeadlines2   
4                                   Hindi States      @HindiStates   

                  Timestamp  Verified  \
0  2024-07-01T13:49:34.000Z     False   
1  2024-07-01T13:17:17.000Z     False   
2  2024-07-01T12:55:41.000Z     False   
3  2024-07-01T12:40:23.000Z     False   
4  2024-07-01T11:58:45.000Z      True   

                                             Content  Comments  Retweets  \
0  #NaturalDisasterViolent #storms and #heavyrain...         0         0   
1  Hurricane Beryl Threatens Caribbean Islandshtt...         0         0   
2  Hurricane Beryl Explosively Intensified - What...         0         0   
3  शहर बने समंदर, उत्तराखंड 

**1.2. Inspect the Data**

Check for missing values and data types to understand what kind of cleaning is required.



In [7]:
# Check for missing values
print(data.isnull().sum())

# Check data types
print(data.dtypes)


Name             5
Handle           0
Timestamp        0
Verified         0
Content          0
Comments         0
Retweets         0
Likes            0
Analytics        0
Tags             0
Mentions         0
Emojis           0
Profile Image    0
Tweet Link       0
Tweet ID         0
dtype: int64
Name             object
Handle           object
Timestamp        object
Verified           bool
Content          object
Comments          int64
Retweets          int64
Likes             int64
Analytics        object
Tags             object
Mentions         object
Emojis           object
Profile Image    object
Tweet Link       object
Tweet ID         object
dtype: object


Based on the inspection results, we need to address the missing values in the Name and Content columns, and also correct the data types for the Timestamp, Likes, and potentially other columns.

**1.3 Handle Missing Values**

In [8]:
# Drop rows with missing 'Content'
data = data.dropna(subset=['Content'])

# Fill missing 'Name' with a placeholder
data.loc[data['Name'].isnull(), 'Name'] = 'Unknown'
data

,Name,Handle,Timestamp,Verified,Content,Comments,Retweets,Likes,Analytics,Tags,Mentions,Emojis,Profile Image,Tweet Link,Tweet ID
0,Eco Discussion,@EcoDiscussion,2024-07-01T13:49:34.000Z,False,#NaturalDisasterViolent #storms and #heavyrain...,0,0,1,31,"['#NaturalDisaster', '#storms', '#heavyrains',...",[],[],https://pbs.twimg.com/profile_images/169178284...,https://x.com/EcoDiscussion/status/18077735464...,tweet_id:1807773546414039212
1,Daily News - Trending | Breaking | Technology,@mirror7news,2024-07-01T13:17:17.000Z,False,Hurricane Beryl Threatens Caribbean Islandshtt...,0,0,0,294,"['#HurricaneBeryl', '#CaribbeanThreat', '#Stor...",[],[],https://pbs.twimg.com/profile_images/163442045...,https://x.com/mirror7news/status/1807765421506...,tweet_id:1807765421506064608
2,Shohag,@MdShohi70476789,2024-07-01T12:55:41.000Z,False,Hurricane Beryl Explosively Intensified - What...,0,0,0,65,"['#BerylIntensifies', '#HurricaneAlert', '#Sta...",[],[],https://pbs.twimg.com/profile_images/163493508...,https://x.com/MdShohi70476789/status/180775998...,tweet_id:1807759984677245142
3,City Headlines,@CityHeadlines2,2024-07-01T12:40:23.000Z,False,"शहर बने समंदर, उत्तराखंड से गुजरात तक मचा हाहा...",0,0,0,12,"['#naturaldisaster', '#uttarakhand', '#gujarat']",[],[],https://pbs.twimg.com/profile_images/175874295...,https://x.com/CityHeadlines2/status/1807756136...,tweet_id:1807756136017203525
4,Hindi States,@HindiStates,2024-07-01T11:58:45.000Z,True,Uttarakhand: #Kedarnath में ग्लेशियर टूटने से ...,0,0,0,27,"['#Kedarnath', '#Uttarakhand', '#GlacierBurst'...",[],[],https://pbs.twimg.com/profile_images/180009706...,https://x.com/HindiStates/status/1807745655302...,tweet_id:1807745655302963350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,News18,@CNNnews18,2024-02-15T13:53:56.000Z,True,Is Iceland entering a new volcanic era? Scient...,0,1,3,1.4K,"['#Iceland', '#Volcano', '#VolcanicEruption', ...",[],[],https://pbs.twimg.com/profile_images/155978869...,https://x.com/CNNnews18/status/175812750687462...,tweet_id:1758127506874621996
1996,Asha Fassbender,@FassbenderENG,2024-02-15T13:23:05.000Z,False,Rescue crews train for earthquake scenario at ...,0,0,0,49,"['#NaturalDisaster', '#NaturalDisaster', '#Dis...",[],[],https://pbs.twimg.com/profile_images/156550577...,https://x.com/FassbenderENG/status/17581197432...,tweet_id:1758119743264014731
1997,Outlook India,@Outlookindia,2024-02-15T11:30:00.000Z,True,#InPhotos | Persistent rainfall triggers flood...,0,0,0,330,"['#InPhotos', '#Floods', '#Turkey', '#NaturalD...",[],[],https://pbs.twimg.com/profile_images/157368479...,https://x.com/Outlookindia/status/175809128566...,tweet_id:1758091285662183850
1998,WILPF,@WILPF,2024-02-15T11:22:23.000Z,False,"#BeyondSurvival— After #SyriaTurkeyEarthquake,...",0,3,2,743,"['#BeyondSurvival', '#SyriaTurkeyEarthquake', ...",[],"['\\U0001f54a\\ufe0f', '\\U0001f31f', '\\u25b6...",https://pbs.twimg.com/profile_images/152614361...,https://x.com/WILPF/status/1758089368265208103,tweet_id:1758089368265208103


In [9]:
# Iterate through the DataFrame and clean the Content column
data['Content'] = data['Content'].apply(lambda x: re.sub(r'http\S+|@\S+|#\S+', '', x) if isinstance(x, str) else x)
data['Content'] = data['Content'].apply(lambda x: re.sub(r'[^\w\s]', '', x) if isinstance(x, str) else x)
data

,Name,Handle,Timestamp,Verified,Content,Comments,Retweets,Likes,Analytics,Tags,Mentions,Emojis,Profile Image,Tweet Link,Tweet ID
0,Eco Discussion,@EcoDiscussion,2024-07-01T13:49:34.000Z,False,and have triggered massive and dramatic fl...,0,0,1,31,"['#NaturalDisaster', '#storms', '#heavyrains',...",[],[],https://pbs.twimg.com/profile_images/169178284...,https://x.com/EcoDiscussion/status/18077735464...,tweet_id:1807773546414039212
1,Daily News - Trending | Breaking | Technology,@mirror7news,2024-07-01T13:17:17.000Z,False,Hurricane Beryl Threatens Caribbean Islands ...,0,0,0,294,"['#HurricaneBeryl', '#CaribbeanThreat', '#Stor...",[],[],https://pbs.twimg.com/profile_images/163442045...,https://x.com/mirror7news/status/1807765421506...,tweet_id:1807765421506064608
2,Shohag,@MdShohi70476789,2024-07-01T12:55:41.000Z,False,Hurricane Beryl Explosively Intensified Whats...,0,0,0,65,"['#BerylIntensifies', '#HurricaneAlert', '#Sta...",[],[],https://pbs.twimg.com/profile_images/163493508...,https://x.com/MdShohi70476789/status/180775998...,tweet_id:1807759984677245142
3,City Headlines,@CityHeadlines2,2024-07-01T12:40:23.000Z,False,शहर बन समदर उततरखड स गजरत तक मच हहकर,0,0,0,12,"['#naturaldisaster', '#uttarakhand', '#gujarat']",[],[],https://pbs.twimg.com/profile_images/175874295...,https://x.com/CityHeadlines2/status/1807756136...,tweet_id:1807756136017203525
4,Hindi States,@HindiStates,2024-07-01T11:58:45.000Z,True,Uttarakhand म गलशयर टटन स मच हलचल परशसन न क त...,0,0,0,27,"['#Kedarnath', '#Uttarakhand', '#GlacierBurst'...",[],[],https://pbs.twimg.com/profile_images/180009706...,https://x.com/HindiStates/status/1807745655302...,tweet_id:1807745655302963350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,News18,@CNNnews18,2024-02-15T13:53:56.000Z,True,Is Iceland entering a new volcanic era Scienti...,0,1,3,1.4K,"['#Iceland', '#Volcano', '#VolcanicEruption', ...",[],[],https://pbs.twimg.com/profile_images/155978869...,https://x.com/CNNnews18/status/175812750687462...,tweet_id:1758127506874621996
1996,Asha Fassbender,@FassbenderENG,2024-02-15T13:23:05.000Z,False,Rescue crews train for earthquake scenario at ...,0,0,0,49,"['#NaturalDisaster', '#NaturalDisaster', '#Dis...",[],[],https://pbs.twimg.com/profile_images/156550577...,https://x.com/FassbenderENG/status/17581197432...,tweet_id:1758119743264014731
1997,Outlook India,@Outlookindia,2024-02-15T11:30:00.000Z,True,Persistent rainfall triggers floods in Turke...,0,0,0,330,"['#InPhotos', '#Floods', '#Turkey', '#NaturalD...",[],[],https://pbs.twimg.com/profile_images/157368479...,https://x.com/Outlookindia/status/175809128566...,tweet_id:1758091285662183850
1998,WILPF,@WILPF,2024-02-15T11:22:23.000Z,False,After Syrian refugees faced more than the b...,0,3,2,743,"['#BeyondSurvival', '#SyriaTurkeyEarthquake', ...",[],"['\\U0001f54a\\ufe0f', '\\U0001f31f', '\\u25b6...",https://pbs.twimg.com/profile_images/152614361...,https://x.com/WILPF/status/1758089368265208103,tweet_id:1758089368265208103


Remove symbols

**1.4 Correct Data Types**

 **Filter for English Language**

In [10]:
# Ensure all Content values are strings
data['Content'] = data['Content'].astype(str)

# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return None

# Apply language detection to the 'Content' column
data['language'] = data['Content'].apply(detect_language)

# Filter out non-English rows
df_english = data[data['language'] == 'en']

# Drop the 'language' column if not needed
df_english = df_english.drop(columns=['language'])

In [11]:
df_english

,Name,Handle,Timestamp,Verified,Content,Comments,Retweets,Likes,Analytics,Tags,Mentions,Emojis,Profile Image,Tweet Link,Tweet ID
0,Eco Discussion,@EcoDiscussion,2024-07-01T13:49:34.000Z,False,and have triggered massive and dramatic fl...,0,0,1,31,"['#NaturalDisaster', '#storms', '#heavyrains',...",[],[],https://pbs.twimg.com/profile_images/169178284...,https://x.com/EcoDiscussion/status/18077735464...,tweet_id:1807773546414039212
1,Daily News - Trending | Breaking | Technology,@mirror7news,2024-07-01T13:17:17.000Z,False,Hurricane Beryl Threatens Caribbean Islands ...,0,0,0,294,"['#HurricaneBeryl', '#CaribbeanThreat', '#Stor...",[],[],https://pbs.twimg.com/profile_images/163442045...,https://x.com/mirror7news/status/1807765421506...,tweet_id:1807765421506064608
2,Shohag,@MdShohi70476789,2024-07-01T12:55:41.000Z,False,Hurricane Beryl Explosively Intensified Whats...,0,0,0,65,"['#BerylIntensifies', '#HurricaneAlert', '#Sta...",[],[],https://pbs.twimg.com/profile_images/163493508...,https://x.com/MdShohi70476789/status/180775998...,tweet_id:1807759984677245142
7,Asianet Newsable,@AsianetNewsEN,2024-07-01T10:17:18.000Z,True,Assam Flood Crisis Deepens 671 Villages Under ...,0,0,2,157,"['#AssamFloods', '#FloodCrisis', '#DisasterMan...",[],[],https://pbs.twimg.com/profile_images/175509069...,https://x.com/AsianetNewsEN/status/18077201247...,tweet_id:1807720124792664456
9,MyShorts,@ShortsMalaysia,2024-07-01T08:06:31.000Z,True,Tragic flood in 1 dead over 100 displaced Fem...,0,0,0,31,"['#PenampangDistrict', '#PenampangFlood', '#Di...",[],['\\U0001f30a'],https://pbs.twimg.com/profile_images/178020774...,https://x.com/ShortsMalaysia/status/1807687211...,tweet_id:1807687211984597025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,News18,@CNNnews18,2024-02-15T13:53:56.000Z,True,Is Iceland entering a new volcanic era Scienti...,0,1,3,1.4K,"['#Iceland', '#Volcano', '#VolcanicEruption', ...",[],[],https://pbs.twimg.com/profile_images/155978869...,https://x.com/CNNnews18/status/175812750687462...,tweet_id:1758127506874621996
1996,Asha Fassbender,@FassbenderENG,2024-02-15T13:23:05.000Z,False,Rescue crews train for earthquake scenario at ...,0,0,0,49,"['#NaturalDisaster', '#NaturalDisaster', '#Dis...",[],[],https://pbs.twimg.com/profile_images/156550577...,https://x.com/FassbenderENG/status/17581197432...,tweet_id:1758119743264014731
1997,Outlook India,@Outlookindia,2024-02-15T11:30:00.000Z,True,Persistent rainfall triggers floods in Turke...,0,0,0,330,"['#InPhotos', '#Floods', '#Turkey', '#NaturalD...",[],[],https://pbs.twimg.com/profile_images/157368479...,https://x.com/Outlookindia/status/175809128566...,tweet_id:1758091285662183850
1998,WILPF,@WILPF,2024-02-15T11:22:23.000Z,False,After Syrian refugees faced more than the b...,0,3,2,743,"['#BeyondSurvival', '#SyriaTurkeyEarthquake', ...",[],"['\\U0001f54a\\ufe0f', '\\U0001f31f', '\\u25b6...",https://pbs.twimg.com/profile_images/152614361...,https://x.com/WILPF/status/1758089368265208103,tweet_id:1758089368265208103


 1.4.1 Convert Data Types

Convert the Timestamp to datetime format, and Likes to integers (handling any non-numeric values).

In [12]:
# Convert 'Timestamp' to datetime
df_english['Timestamp'] = pd.to_datetime(df_english['Timestamp'])

# Clean and convert 'Likes' to numeric, fill non-numeric values with 0
df_english['Likes'] = pd.to_numeric(df_english['Likes'], errors='coerce').fillna(0).astype(int)


In [13]:
df_english.dtypes

,0
Name,object
Handle,object
Timestamp,"datetime64[ns, UTC]"
Verified,bool
Content,object
Comments,int64
Retweets,int64
Likes,int64
Analytics,object
Tags,object


**Remove Unnecessary Columns**

Remove columns that are not needed for the analysis (Analytics, Profile Image, Tweet Link, Emojis).

In [14]:
# Remove unnecessary columns
columns_to_keep = ['Name', 'Timestamp', 'Content', 'Comments', 'Retweets', 'Likes', 'Tags', 'Mentions','Tweet ID','Handle']
df_english = df_english[columns_to_keep]


**Data normalization**

In [15]:
def normalize_text(text):
    # Lowercase text
    text = text.lower()
    # Remove special characters
    text = ''.join([char for char in text if char.isalnum() or char == ' '])
    # Tokenization and lemmatization
    tokens = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in tokens if not token.is_stop]
    return ' '.join(lemmatized_tokens)

**Load SpaCy model**


In [16]:
# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

**Perform Sentiment Analysis**

In [17]:
# Function to perform sentiment analysis
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

In [18]:
# Function to check for the presence of specific keywords
def check_keywords(text, keywords):
    for keyword in keywords:
        if keyword in text.lower():
            return True
    return False

**NewsAPI Key and Fetch Function**




In [19]:
# API key from NewsAPI
news_api_key = 'cbeb98abd1314723843a0af7d33bb261'

In [20]:
# Function to fetch news articles
def fetch_news(query):
    base_url = 'https://newsapi.org/v2/everything'
    params = {
        'q': query,
        'language': 'en',
        'sortBy': 'relevance',
        'apiKey': news_api_key
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()['articles']
    else:
        print(f"Failed to fetch news articles: {response.status_code}")
        return []

**Fetch Relevant News Articles**

In [21]:
# Fetch news articles related to 'natural disaster'
articles = fetch_news('natural disaster')

In [26]:
# Extract titles and descriptions from the fetched articles
news_contents = []
for article in articles:
    if article['title']:
        news_contents.append(article['title'])
    if article['description']:
        news_contents.append(article['description'])

In [27]:
# Preprocess text for better matching
def preprocess_text(text):
    doc = nlp(text.lower())
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])


In [28]:
# Preprocess news contents
preprocessed_news = [preprocess_text(content) for content in news_contents]


**Verification Function**

In [24]:
# Function to compare tweet with news articles using cosine similarity
def verify_tweet(tweet, preprocessed_news):
    tweet_processed = preprocess_text(tweet)
    documents = [tweet_processed] + preprocessed_news
    tfidf_vectorizer = TfidfVectorizer().fit_transform(documents)
    cosine_similarities = cosine_similarity(tfidf_vectorizer[0:1], tfidf_vectorizer[1:]).flatten()
    max_similarity = cosine_similarities.max()
    return max_similarity > 0.3  # Adjust threshold as needed


In [29]:
# Add columns for sentiment analysis, false info check, and verification against
false_info_keywords = ['false', 'misleading', 'untrue', 'hoax', 'fake news']
df_english['Sentiment'] = df_english['Content'].apply(get_sentiment)
df_english['Contains False Info'] = df_english['Content'].apply(lambda x: check_keywords(x, false_info_keywords))
df_english['Verify'] = df_english['Content'].apply(lambda x: verify_tweet(x, preprocessed_news))
df_english


/tmp/ipython-input-29-442525426.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['Sentiment'] = df_english['Content'].apply(get_sentiment)
/tmp/ipython-input-29-442525426.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['Contains False Info'] = df_english['Content'].apply(lambda x: check_keywords(x, false_info_keywords))
/tmp/ipython-input-29-442525426.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

,Name,Timestamp,Content,Comments,Retweets,Likes,Tags,Mentions,Tweet ID,Handle,Sentiment,Contains False Info,Verify
0,Eco Discussion,2024-07-01 13:49:34+00:00,and have triggered massive and dramatic fl...,0,0,1,"['#NaturalDisaster', '#storms', '#heavyrains',...",[],tweet_id:1807773546414039212,@EcoDiscussion,-0.059722,False,False
1,Daily News - Trending | Breaking | Technology,2024-07-01 13:17:17+00:00,Hurricane Beryl Threatens Caribbean Islands ...,0,0,0,"['#HurricaneBeryl', '#CaribbeanThreat', '#Stor...",[],tweet_id:1807765421506064608,@mirror7news,0.000000,False,False
2,Shohag,2024-07-01 12:55:41+00:00,Hurricane Beryl Explosively Intensified Whats...,0,0,0,"['#BerylIntensifies', '#HurricaneAlert', '#Sta...",[],tweet_id:1807759984677245142,@MdShohi70476789,0.000000,False,True
7,Asianet Newsable,2024-07-01 10:17:18+00:00,Assam Flood Crisis Deepens 671 Villages Under ...,0,0,2,"['#AssamFloods', '#FloodCrisis', '#DisasterMan...",[],tweet_id:1807720124792664456,@AsianetNewsEN,0.000000,False,False
9,MyShorts,2024-07-01 08:06:31+00:00,Tragic flood in 1 dead over 100 displaced Fem...,0,0,0,"['#PenampangDistrict', '#PenampangFlood', '#Di...",[],tweet_id:1807687211984597025,@ShortsMalaysia,-0.256250,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,News18,2024-02-15 13:53:56+00:00,Is Iceland entering a new volcanic era Scienti...,0,1,3,"['#Iceland', '#Volcano', '#VolcanicEruption', ...",[],tweet_id:1758127506874621996,@CNNnews18,0.136364,False,False
1996,Asha Fassbender,2024-02-15 13:23:05+00:00,Rescue crews train for earthquake scenario at ...,0,0,0,"['#NaturalDisaster', '#NaturalDisaster', '#Dis...",[],tweet_id:1758119743264014731,@FassbenderENG,0.000000,False,False
1997,Outlook India,2024-02-15 11:30:00+00:00,Persistent rainfall triggers floods in Turke...,0,0,0,"['#InPhotos', '#Floods', '#Turkey', '#NaturalD...",[],tweet_id:1758091285662183850,@Outlookindia,0.000000,False,False
1998,WILPF,2024-02-15 11:22:23+00:00,After Syrian refugees faced more than the b...,0,3,2,"['#BeyondSurvival', '#SyriaTurkeyEarthquake', ...",[],tweet_id:1758089368265208103,@WILPF,0.150000,False,False


**Analysis of Public Engagement**

In [30]:
# Calculate engagement metrics
df_english['Engagement'] = df_english['Likes'] + df_english['Retweets'] + df_english['Comments']

# Group by verification status
engagement_stats = df_english.groupby('Verify')['Engagement'].describe()
print(engagement_stats)

         count      mean        std  min  25%  50%  75%    max
Verify                                                        
False   1510.0  2.407285  23.113121  0.0  0.0  0.0  1.0  862.0
True      70.0  1.142857   2.168855  0.0  0.0  0.0  1.0    8.0


/tmp/ipython-input-30-926625064.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['Engagement'] = df_english['Likes'] + df_english['Retweets'] + df_english['Comments']


**Engagement Analysis for Specific Users**

In [31]:
import random

# Select a random user
random_user = random.choice(df_english['Handle'].unique())

# Analyze engagement for the selected user
user_tweets = df_english[df_english['Handle'] == random_user]

user_engagement_stats = user_tweets.groupby('Verify')['Engagement'].describe()
print(f"Engagement stats for user {random_user}:")
print(user_engagement_stats)


Engagement stats for user @DecillionGoal:
        count      mean       std  min  25%  50%  75%  max
Verify                                                    
False     3.0  4.666667  2.081666  3.0  3.5  4.0  5.5  7.0


**Save Cleaned Data**

In [32]:
# Save the results to a new CSV file
df_english.to_csv('tweet_result.csv', index=False)

In [33]:
true_twe = df_english[df_english['Verify'] == True]
print("Number of tweets containing TRUE information:", len(true_twe))

Number of tweets containing TRUE information: 70


In [34]:
df_english

,Name,Timestamp,Content,Comments,Retweets,Likes,Tags,Mentions,Tweet ID,Handle,Sentiment,Contains False Info,Verify,Engagement
0,Eco Discussion,2024-07-01 13:49:34+00:00,and have triggered massive and dramatic fl...,0,0,1,"['#NaturalDisaster', '#storms', '#heavyrains',...",[],tweet_id:1807773546414039212,@EcoDiscussion,-0.059722,False,False,1
1,Daily News - Trending | Breaking | Technology,2024-07-01 13:17:17+00:00,Hurricane Beryl Threatens Caribbean Islands ...,0,0,0,"['#HurricaneBeryl', '#CaribbeanThreat', '#Stor...",[],tweet_id:1807765421506064608,@mirror7news,0.000000,False,False,0
2,Shohag,2024-07-01 12:55:41+00:00,Hurricane Beryl Explosively Intensified Whats...,0,0,0,"['#BerylIntensifies', '#HurricaneAlert', '#Sta...",[],tweet_id:1807759984677245142,@MdShohi70476789,0.000000,False,True,0
7,Asianet Newsable,2024-07-01 10:17:18+00:00,Assam Flood Crisis Deepens 671 Villages Under ...,0,0,2,"['#AssamFloods', '#FloodCrisis', '#DisasterMan...",[],tweet_id:1807720124792664456,@AsianetNewsEN,0.000000,False,False,2
9,MyShorts,2024-07-01 08:06:31+00:00,Tragic flood in 1 dead over 100 displaced Fem...,0,0,0,"['#PenampangDistrict', '#PenampangFlood', '#Di...",[],tweet_id:1807687211984597025,@ShortsMalaysia,-0.256250,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,News18,2024-02-15 13:53:56+00:00,Is Iceland entering a new volcanic era Scienti...,0,1,3,"['#Iceland', '#Volcano', '#VolcanicEruption', ...",[],tweet_id:1758127506874621996,@CNNnews18,0.136364,False,False,4
1996,Asha Fassbender,2024-02-15 13:23:05+00:00,Rescue crews train for earthquake scenario at ...,0,0,0,"['#NaturalDisaster', '#NaturalDisaster', '#Dis...",[],tweet_id:1758119743264014731,@FassbenderENG,0.000000,False,False,0
1997,Outlook India,2024-02-15 11:30:00+00:00,Persistent rainfall triggers floods in Turke...,0,0,0,"['#InPhotos', '#Floods', '#Turkey', '#NaturalD...",[],tweet_id:1758091285662183850,@Outlookindia,0.000000,False,False,0
1998,WILPF,2024-02-15 11:22:23+00:00,After Syrian refugees faced more than the b...,0,3,2,"['#BeyondSurvival', '#SyriaTurkeyEarthquake', ...",[],tweet_id:1758089368265208103,@WILPF,0.150000,False,False,5


**Adds new columns to the DataFrame**

1.   Sentiment: Sentiment polarity of each tweet.

2.   Contains False Info: Boolean indicating whether the tweet contains any keywords suggesting false information.
3.  Verified: Boolean indicating whether the tweet content matches any fetched news articles.






**Why Each Step is Performed**



1.   **Sentiment Analysis**: Helps understand the emotional
tone of the tweets, which can be useful for identifying potentially alarming or misleading content. Sentiment analysis can provide additional context to the nature of the tweets being analyzed.
2.   **Keyword Check**: Identifies tweets that may contain false information based on specific keywords. This step is crucial for flagging tweets that need further verification.
3. **NewsAPI Integration**: Fetching news articles from reliable sources allows the script to cross-check the content of tweets with verified information. This helps in verifying the authenticity of the tweets.
4. **Data Saving**: Saving the results to a new CSV file ensures that the analysis can be reviewed and used later without rerunning the script.

This structured approach helps in systematically analyzing and verifying the authenticity of tweets related to natural disasters, combining sentiment analysis, keyword checking, and content verification against reliable news sources.









